In [ ]:
import numpy as np
import pandas as pd
import pydicom as dcm
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
## load dicom image file for both positive and negative instances
pos_dcm = dcm.dcmread("pos.dcm")
neg_dcm = dcm.dcmread("neg.dcm")

In [ ]:
def dcm_bit_standardize(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x >= px_mode] = x[x >= px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

In [ ]:
def window_image(dcm, window_center, window_width):
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        print("Bit Standardization")
        dcm_bit_standardize(dcm)

    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)

    return img

### Brain Window Visualization

#### Positive Instance

In [ ]:
## visualize dicom image instance in brain window
pos_brain = window_image(pos_dcm, 40, 80)
plt.imshow(pos_brain, cmap=plt.cm.gray)

In [ ]:
## visualize normalized brain window image
norm_pos_brain = (pos_brain - 0) / 80
plt.imshow(norm_pos_brain, cmap=plt.cm.gray)

#### Negative Instance

In [ ]:
## visualize dicom image instance in brain window
neg_brain = window_image(neg_dcm, 40, 80)
plt.imshow(neg_brain, cmap=plt.cm.gray)

In [ ]:
## visualize normalized brain window image
norm_neg_brain = (neg_brain - 0) / 80
plt.imshow(norm_neg_brain, cmap=plt.cm.gray)

### Subdural Window Visualization

#### Positive Instance

In [ ]:
## visualize dicom image instance in subdural window
pos_subdural = window_image(pos_dcm, 80, 200)
plt.imshow(pos_subdural, cmap=plt.cm.gray)

In [ ]:
## visualize normalized subdural window image
norm_pos_subdural = (pos_subdural - (-20)) / 200
plt.imshow(norm_pos_subdural, cmap=plt.cm.gray)

#### Negative Instance

In [ ]:
## visualize dicom image instance in subdural window
neg_subdural = window_image(neg_dcm, 80, 200)
plt.imshow(neg_subdural, cmap=plt.cm.gray)

In [ ]:
## visualize normalized subdural window image
norm_neg_subdural = (neg_subdural - (-20)) / 200
plt.imshow(norm_neg_subdural, cmap=plt.cm.gray)

### Soft Tissue Window Visualization

#### Positive Instance

In [ ]:
## visualize dicom image instance in soft tissue window
pos_soft = window_image(pos_dcm, 40, 380)
plt.imshow(pos_soft, cmap=plt.cm.gray)

In [ ]:
## visualize normalized soft tissue window image
norm_pos_soft= (pos_soft - (-150)) / 380
plt.imshow(norm_pos_soft, cmap=plt.cm.gray)

#### Negative Instance

In [ ]:
## visualize dicom image instance in soft tissue window
neg_soft = window_image(neg_dcm, 40, 380)
plt.imshow(neg_soft, cmap=plt.cm.gray)

In [ ]:
## visualize normalized soft tissue window image
norm_neg_soft= (neg_soft - (-150)) / 380
plt.imshow(norm_neg_soft, cmap=plt.cm.gray)

In [ ]:
def channel_combine(dcm):
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    soft_img = window_image(dcm, 40, 380)

    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380
    
    combined_img = np.array([brain_img, subdural_img, soft_img]).transpose(1, 2, 0)

    return combined_img

### Positive Instance

In [ ]:
## Visualize the combined channel Image
pos_combined = channel_combine(pos_dcm)
plt.imshow(pos_combined, cmap=plt.cm.gray)

### Negative Instance

In [ ]:
## Visualize the combined channel Image
neg_combined = channel_combine(neg_dcm)
plt.imshow(neg_combined, cmap=plt.cm.gray)

### Familiar Instance

In [ ]:
test_dcm = dcm.dcmread("test.dcm")

### Brain Window

In [ ]:
## visualize dicom image instance in brain window
test_brain = window_image(test_dcm, 40, 80)
plt.imshow(test_brain, cmap=plt.cm.gray)

In [ ]:
## visualize normalized brain window image
norm_test_brain = (test_brain - 0) / 80
plt.imshow(norm_test_brain, cmap=plt.cm.gray)

### Subdural Window

In [ ]:
## visualize dicom image instance in subdural window
test_subdural = window_image(test_dcm, 80, 200)
plt.imshow(test_subdural, cmap=plt.cm.gray)

In [ ]:
## visualize normalized subdural window image
norm_test_subdural = (test_subdural - (-20)) / 200
plt.imshow(norm_test_subdural, cmap=plt.cm.gray)

### Soft Tissue Window

In [ ]:
## visualize dicom image instance in soft tissue window
test_soft = window_image(test_dcm, 40, 380)
plt.imshow(test_soft, cmap=plt.cm.gray)

In [ ]:
## visualize normalized soft tissue window image
norm_test_soft= (test_soft - (-150)) / 380
plt.imshow(norm_test_soft, cmap=plt.cm.gray)

### Combined Channel

In [ ]:
## Visualize the combined channel Image
test_combined = channel_combine(test_dcm)
plt.imshow(test_combined, cmap=plt.cm.gray)

#### Image Blurring - Gaussian Blurring

In [ ]:
from PIL import Image, ImageFilter 

In [ ]:
from scipy.ndimage.filters import gaussian_filter
brain_blurred = gaussian_filter(norm_test_brain, sigma=4)
soft_blurred = gaussian_filter(norm_test_soft, sigma=4)
subdural_blurred = gaussian_filter(norm_test_subdural, sigma=4)

In [ ]:
combined_blurred = np.array([brain_blurred, subdural_blurred, soft_blurred]).transpose(1, 2, 0)

In [ ]:
plt.imshow(combined_blurred)